In [12]:
import statsapi
sched = statsapi.schedule(start_date='01/01/2024',end_date='12/30/2024')
# team id key dict
t_info = {}

from team import team
t = team()
t.getAll()
for tt in t.data:
    t_info[tt["teamid"]] = tt['teamKey']
    
game_list = []
for i in sched:
    if i['status'] == 'Final' and i['game_type'] == "R":
        game = {}
        game['gameid'] = i['game_id']
        game['date'] = i['game_date']
        game['awayTeam'] = i['away_name']
        game['homeTeam'] = i['home_name']
        game['awayKey'] = t_info[i['away_id']]
        game['homeKey'] = t_info[i['home_id']]
        game['awayid'] = i['away_id']
        game['homeid'] = i['home_id']
        game['homeScore'] = i['home_score']
        game['awayScore'] = i['away_score']
        game_list.append(game)

In [17]:
import statsapi
from batterbox import batterbox
b = batterbox()
b.truncate()
count = 0
sql = f"INSERT INTO `{b.tn}` ("
for field in b.fields:
    sql += f"`{field}`,"
    count +=1
sql = sql[0:-1] + ') VALUES ('
tokens = ("%s," * count)[0:-1]
sql += tokens + ');'

for g in game_list:
    df = statsapi.boxscore_data(g['gameid'])
    p_info = []
    n = 0
    for k in df['awayBatters']:
        if n > 0:
            p = {}
            p['gameid'] = g['gameid']
            p['hw'] = 'away'
            p['teamid'] = df['teamInfo']['away']['id']
            p['abbreviation'] = df['teamInfo']['away']['abbreviation']
            p['batterName'] = k['name']
            p['batterid'] = k['personId']
            p['position'] = k['position']
            p['battingOrder'] = k['battingOrder']
            p['AB'] = k['ab']
            p['R'] = k['r']
            p['H'] = k['h']
            p['RBI'] = k['rbi']
            p['BB'] = k['bb']
            p['K'] = k['k']
            p['AVG'] = k['avg']
            p['OPS'] = k['ops']
            p_info.append(p)
        n += 1
    away_total = {}
    away_total['gameid'] = g['gameid']
    away_total['hw'] = 'away'
    away_total['teamid'] = df['teamInfo']['away']['id']
    away_total['abbreviation'] = df['teamInfo']['away']['abbreviation']
    away_total['batterName'] = df['awayBattingTotals']['name']
    away_total['batterid'] = 0
    away_total['position'] = ''
    away_total['battingOrder'] = 0
    away_total['AB'] = df['awayBattingTotals']['ab']
    away_total['R'] = df['awayBattingTotals']['r']
    away_total['H'] = df['awayBattingTotals']['h']
    away_total['RBI'] = df['awayBattingTotals']['rbi']
    away_total['BB'] = df['awayBattingTotals']['bb']
    away_total['K'] = df['awayBattingTotals']['k']
    away_total['AVG'] = 0
    away_total['OPS'] = 0
    p_info.append(away_total)
    n = 0
    for k in df['homeBatters']:
        if n > 0:
            p = {}
            p['gameid'] = g['gameid']
            p['hw'] = 'home'
            p['teamid'] = df['teamInfo']['home']['id']
            p['abbreviation'] = df['teamInfo']['home']['abbreviation']
            p['batterName'] = k['name']
            p['batterid'] = k['personId']
            p['position'] = k['position']
            p['battingOrder'] = k['battingOrder']
            p['AB'] = k['ab']
            p['R'] = k['r']
            p['H'] = k['h']
            p['RBI'] = k['rbi']
            p['BB'] = k['bb']
            p['K'] = k['k']
            p['AVG'] = k['avg']
            p['OPS'] = k['ops']
            p_info.append(p)
        n += 1
    home_total = {}
    home_total['gameid'] = g['gameid']
    home_total['hw'] = 'home'
    home_total['teamid'] = df['teamInfo']['home']['id']
    home_total['abbreviation'] = df['teamInfo']['home']['abbreviation']
    home_total['batterName'] = df['homeBattingTotals']['name']
    home_total['batterid'] = 0
    home_total['position'] = ''
    home_total['battingOrder'] = 0
    home_total['AB'] = df['homeBattingTotals']['ab']
    home_total['R'] = df['homeBattingTotals']['r']
    home_total['H'] = df['homeBattingTotals']['h']
    home_total['RBI'] = df['homeBattingTotals']['rbi']
    home_total['BB'] = df['homeBattingTotals']['bb']
    home_total['K'] = df['homeBattingTotals']['k']
    home_total['AVG'] = 0
    home_total['OPS'] = 0
    p_info.append(home_total)

    vals = []
    for p in p_info:
        val = []
        for field in b.fields:
            val.append(p[field])
        vals.append(val) 

    b.cur.executemany(sql,vals)



In [18]:
b.getByMultiField(["gameid",'hw'], [745373,'away'])